In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# handle gpu leakage issue on DSI cluster
# ideally we shouldn't have to do this but leaving it for now in case the issue pops up again
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [92]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaForCausalLM, LlamaTokenizer
from transformers.modeling_outputs import CausalLMOutputWithPast

import scripts

In [5]:
# Confirm GPUs are working properly and set default device explicitly
device = "cuda:0"
torch.cuda.device_count()

1

In [6]:
MODEL_DIR = "/net/projects/veitch/LLMs/"
MODEL = "llama1-based-models/alpaca-7b" # or llama2-based-models/llama2-hf/Llama-2-7b-chat-hf
MODEL_PATH = MODEL_DIR + MODEL

In [6]:
# TODO: Clean this up

# This cell is for doing ROME edits
# ROME model and tokenizer
# MODEL_NAME = "gpt2-xl"
# model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# tokenizer.pad_token = tokenizer.eos_token

In [7]:
tokenizer = LlamaTokenizer.from_pretrained(MODEL_PATH)
model = LlamaForCausalLM.from_pretrained(MODEL_PATH, low_cpu_mem_usage=True, torch_dtype=torch.float16, device_map="auto").to(device)

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
LABELS = "title" # choices are "title" and "categories"
DATA_FOLDER = "data/"
TRAIN_FILEPATH = DATA_FOLDER + "train-12-articles.csv"
VAL_FILEPATH = DATA_FOLDER + "val-12-articles.csv"
PROBE_TYPE = "linear" # choices are "linear" and "mlp"
LAYER = None
EPOCHS = 50
BATCH_SIZE = 4
AGGREGATION = "max" # choices are "max" or "mean"
PRINT_PROGRESS = True

In [9]:
train_data, label_encoder_title, label_encoder_cats = scripts.get_hidden_states(TRAIN_FILEPATH, model, tokenizer, device)
val_data, _, _ = scripts.get_hidden_states(VAL_FILEPATH, model, tokenizer, device)

In [14]:
# to use entire flattened activations use layer = None
train_dataset = scripts.create_dataset(*train_data, layer=-1)

In [15]:
val_dataset = scripts.create_dataset(*val_data, layer=-1)

In [16]:
val_dataset.Xs[0].shape

torch.Size([4096])

In [19]:
probe, _ = scripts.train_handler(train_dataset, val_dataset, label_encoder_title, probe_type=PROBE_TYPE, labels=LABELS, batch_size=BATCH_SIZE, epochs=EPOCHS, print_progress=PRINT_PROGRESS)

[Training][5] loss: 1.459
[Validation][5] loss: 0.275
[Validation]5 accuracy: 0.833
[Training][10] loss: 1.276
[Validation][10] loss: 0.412
[Validation]10 accuracy: 0.778
[Training][15] loss: 0.641
[Validation][15] loss: 0.447
[Validation]15 accuracy: 0.806
[Training][20] loss: 0.033
[Validation][20] loss: 0.099
[Validation]20 accuracy: 0.917
[Training][25] loss: 0.000
[Validation][25] loss: 0.098
[Validation]25 accuracy: 0.917
[Training][30] loss: 0.000
[Validation][30] loss: 0.091
[Validation]30 accuracy: 0.917
[Training][35] loss: 0.000
[Validation][35] loss: 0.075
[Validation]35 accuracy: 0.917
[Training][40] loss: 0.000
[Validation][40] loss: 0.068
[Validation]40 accuracy: 0.917
[Training][45] loss: 0.000
[Validation][45] loss: 0.063
[Validation]45 accuracy: 0.944
text:  the part of the American judicial system that is responsible for hearing and reviewing appeals from legal cases that have already been heard in a trial-level or other lower court
labels:  Appellate court
predicted

### Try Editing With Linear Probe

In [24]:
probe.fc1.weight.data.shape

torch.Size([12, 4096])

In [25]:
label_encoder_title.classes_

array(['Agriculture', 'Alaska', 'Alchemy', 'Algae', 'American football',
       'Amphibian', 'Anarchism', 'Animation', 'Anthropology',
       'Appellate court', 'Astronomer', 'Basketball'], dtype='<U17')

In [69]:
prompt = "I hate authority and my favorite form of government is"

input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(device)
input_ids.shape

torch.Size([1, 11])

In [70]:
output = model(input_ids, output_hidden_states=True)
output.hidden_states[-1].shape

torch.Size([1, 11, 4096])

In [75]:
logits = model.lm_head(output.hidden_states[-1])
logits.shape

torch.Size([1, 11, 32001])

In [78]:
logits_mod = logits[:,-1,:] - probe.fc1.weight.data[6]

RuntimeError: The size of tensor a (32001) must match the size of tensor b (4096) at non-singleton dimension 1

In [76]:
next_token = logits[:,-1,:].argmax(-1)
next_token.shape

torch.Size([1])

In [77]:
generated_text = tokenizer.decode(next_token, skip_special_tokens=True)
generated_text

'an'

In [ ]:
# TODO: Try modifying the final layer of the outputs

In [99]:
output = editable_model.generate(input_ids, max_length=50, num_return_sequences=1, output_hidden_states=True) # default search strategy is greedy
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "pad_token_id": 0,
  "transformers_version": "4.31.0"
}



torch.Size([1, 11, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 4096])


In [100]:
generated_text

"I hate authority and my favorite form of government is anarchy.\nI'm a big fan of the idea of a free society, but I'm not sure that anarchy is the way to get there.\nI think that an"

### Try Inheriting Llama

In [88]:
from typing import List, Optional, Tuple, Union

In [97]:
class EditableLlama(LlamaForCausalLM):
    def __init__(self, config, edit=None, *args, **kwargs):
        super().__init__(config, *args, **kwargs)
        self.edit = edit

    def forward(
        self,
        input_ids: torch.LongTensor = None,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[List[torch.FloatTensor]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, CausalLMOutputWithPast]:
        r"""
        Args:
            labels (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
                Labels for computing the masked language modeling loss. Indices should either be in `[0, ...,
                config.vocab_size]` or -100 (see `input_ids` docstring). Tokens with indices set to `-100` are ignored
                (masked), the loss is only computed for the tokens with labels in `[0, ..., config.vocab_size]`.

        Returns:

        Example:

        ```python
        >>> from transformers import AutoTokenizer, LlamaForCausalLM

        >>> model = LlamaForCausalLM.from_pretrained(PATH_TO_CONVERTED_WEIGHTS)
        >>> tokenizer = AutoTokenizer.from_pretrained(PATH_TO_CONVERTED_TOKENIZER)

        >>> prompt = "Hey, are you conscious? Can you talk to me?"
        >>> inputs = tokenizer(prompt, return_tensors="pt")

        >>> # Generate
        >>> generate_ids = model.generate(inputs.input_ids, max_length=30)
        >>> tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
        "Hey, are you conscious? Can you talk to me?\nI'm not conscious, but I can talk to you."
        ```"""

        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        # decoder outputs consists of (dec_features, layer_state, dec_hidden, dec_attn)
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        hidden_states = outputs[0]
        print(hidden_states.shape)
        # TODO: Insert model edit here
        # torch.Size([1, 11, 4096])
        # torch.Size([1, 1, 4096])

        if self.config.pretraining_tp > 1:
            lm_head_slices = self.lm_head.weight.split(self.vocab_size // self.config.pretraining_tp, dim=0)
            logits = [F.linear(hidden_states, lm_head_slices[i]) for i in range(self.config.pretraining_tp)]
            logits = torch.cat(logits, dim=-1)
        else:
            logits = self.lm_head(hidden_states)
        logits = logits.float()

        loss = None
        if labels is not None:
            # Shift so that tokens < n predict n
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            # Flatten the tokens
            loss_fct = CrossEntropyLoss()
            shift_logits = shift_logits.view(-1, self.config.vocab_size)
            shift_labels = shift_labels.view(-1)
            # Enable model parallelism
            shift_labels = shift_labels.to(shift_logits.device)
            loss = loss_fct(shift_logits, shift_labels)

        if not return_dict:
            output = (logits,) + outputs[1:]
            return (loss,) + output if loss is not None else output

        return CausalLMOutputWithPast(
            loss=loss,
            logits=logits,
            past_key_values=outputs.past_key_values,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [98]:
editable_model = EditableLlama.from_pretrained(MODEL_PATH, low_cpu_mem_usage=True, torch_dtype=torch.float16, device_map="auto").to(device)

loading configuration file /net/projects/veitch/LLMs/llama1-based-models/alpaca-7b/config.json
Model config LlamaConfig {
  "_name_or_path": "/net/scratch/reber/alpaca-7b",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.31.0",
  "use_cache": true,
  "vocab_size": 32001
}

loading weights file /net/projects/veitch/LLMs/llama1-based-models/alpaca-7b/pytorch_model.bin.index.json
Instantiating EditableLlama model under default dtype torch.float16.
Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_tok

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing EditableLlama.

All the weights of EditableLlama were initialized from the model checkpoint at /net/projects/veitch/LLMs/llama1-based-models/alpaca-7b.
If your task is similar to the task the model of the checkpoint was trained on, you can already use EditableLlama for predictions without further training.
loading configuration file /net/projects/veitch/LLMs/llama1-based-models/alpaca-7b/generation_config.json
Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "pad_token_id": 0,
  "transformers_version": "4.31.0"
}

